* 這次練習剛好跟 sklearn 的[分類法/範例一: Recognizing hand-written digits](https://machine-learning-python.kspax.io/Classification/ex1_Recognizing_hand-written_digits.html) 資料相似
* 只是一個用tansorflow (softmax + cross_entropy)
* 另一個用sklearn(svc)

In [19]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  #當電腦中沒有這個練習資料庫時，有這行他會幫你下載

#建立數據
xs = tf.placeholder(tf.float32,[None,784])  #資料形式float32，格式None(不規範)， 資料量784(28*28)
ys = tf.placeholder(tf.float32,[None,10])

#定義神經層
def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]))
    baises = tf.Variable(tf.zeros([1,out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs,Weights) + baises
                        #^^^^^^^^^^^^^^^不能顛倒
    if activation_function is None:
        output = Wx_plus_b
    else:
        output = activation_function(Wx_plus_b)
    return output

#建立神經層
prediction_layer = add_layer(xs,784,10, activation_function= tf.nn.softmax) #<---帶入分類法中的softmax

#loss (重點！難！)(crosss_entropy是和softmax配合)
#似乎是向量相乘去減少向量偏差
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction_layer),reduction_indices=[1])) 

#建立優化器
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
               #^^^^^常忘記

#定義準確率(v_xs和v_ys就是每次切割100份量資料丟進去時的資料) #註一
def compute_accuracy(v_xs,v_ys):
    global prediction_layer
    y_pre = sess.run(prediction_layer,feed_dict={xs:v_xs})       #預測的(0~9每格的概率值)
    correct_pre = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1)) #正確的(每格概率值和正確的每格概略值比較，每格輸出bool值)
    accuracy = tf.reduce_mean(tf.cast(correct_pre,tf.float32))   #準確率（cast將bool轉為0,1再計算準確率）
    
    result = sess.run(accuracy, feed_dict={xs:v_xs,ys:v_ys})
    return result

#建立環境和啟動變量
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

#train
for i in range(1000):
    #資料量太大，先切割資料去訓練(就像sklearn.cross_validation中的train_test_split)(batch:批量)
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs:batch_xs,ys:batch_ys})
    #將訓練過程每隔一個區段印出準確度
    if i%50 == 0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.0826
0.6268
0.7392
0.7929
0.8095
0.8269
0.8358
0.8414
0.8467
0.8488
0.857
0.8628
0.8643
0.8647
0.8687
0.8679
0.8723
0.875
0.8747
0.8784


### 註一
评估模型

那么我们的模型性能如何呢？

首先让我们找出那些预测正确的标签。tf.argmax 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值。由于标签向量是由0,1组成，因此最大值1所在的索引位置就是类别标签，比如tf.argmax(y,1)返回的是模型对于任一输入x预测到的标签值，而 tf.argmax(y_,1) 代表正确的标签，我们可以用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)。

>correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

这里返回一个布尔数组。为了计算我们分类的准确率，我们将布尔值转换为浮点数来代表对、错，然后取平均值。例如：[True, False, True, True]变为[1,0,1,1]，计算出平均值为0.75。

>accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))